In [86]:
import sys
from pathlib import Path
from dotenv import load_dotenv
import os

# Load environment variables from the .env file
load_dotenv()

WORKSPACE_PATH = os.getenv("WORKSPACE_PATH")

# Add the parent directory to the system path
sys.path.append(str(WORKSPACE_PATH))


In [87]:
from utils.general_utils import group_object_list

In [88]:
import copy
from datetime import datetime, timedelta


In [89]:
from src.classes import Meas

In [90]:
def adjust_timestamps(ts1, ts2):
    """
    Adjusts two timestamps by setting the older one to zero and the younger one to the older timestamp plus the difference in seconds.

    Parameters:
    ts1 (Union[str, datetime]): The first timestamp in ISO format or a datetime object.
    ts2 (Union[str, datetime]): The second timestamp in ISO format or a datetime object.

    Returns:
    tuple: A tuple containing the adjusted timestamps (older, adjusted_younger).
    """
    # Check if inputs are strings and parse them as datetime objects if they are
    if isinstance(ts1, str):
        timestamp1 = datetime.fromisoformat(ts1)
    elif isinstance(ts1, datetime):
        timestamp1 = ts1
    else:
        raise TypeError("ts1 must be a string or a datetime object.")

    if isinstance(ts2, str):
        timestamp2 = datetime.fromisoformat(ts2)
    elif isinstance(ts2, datetime):
        timestamp2 = ts2
    else:
        raise TypeError("ts2 must be a string or a datetime object.")

    # Determine the older and younger timestamps
    if timestamp1 < timestamp2:
        older = datetime(1970, 1, 1)  # Set the older timestamp to epoch (zero)
        difference = (timestamp2 - timestamp1).total_seconds()  # Difference in seconds
    else:
        older = datetime(1970, 1, 1)  # Set the older timestamp to epoch (zero)
        difference = (timestamp1 - timestamp2).total_seconds()  # Difference in seconds

    # Adjust the younger timestamp by adding the difference to the older timestamp
    adjusted_younger = older + timedelta(seconds=difference)

    return older, adjusted_younger

In [91]:
def normalize_meas_time(meas_list: list[Meas]) -> list[Meas]:
    grouped_meas = group_object_list(
        meas_list,
        ["metadata.meas_number", "metadata.condition", "metadata.pair_number"],
    )

    # Iterate over each group of measurements
    for key, group in grouped_meas.items():
        meas_number, condition, pair_number = key

        # Separate measurements by gender
        person_meas1 = [meas for meas in group if meas.metadata.gender == "M"]
        person_meas2 = [meas for meas in group if meas.metadata.gender == "F"]

        # Ensure both genders are present
        if not person_meas1 or not person_meas2:
            logger.warning(
                f"{meas_number}, {condition}, {pair_number}: Skipping pair. Lack of meas1 or meas2."
            )
            continue

        # Find the oldest starttime among all measurements in the pair
        all_meas = person_meas1 + person_meas2
        oldest_starttime = min(meas.metadata.starttime for meas in all_meas)

        # Set the starttimes of both measurements in the pair to the oldest starttime
        for meas in all_meas:
            durration = meas.metadata.endtime - meas.metadata.starttime
            _, meas.metadata.starttime = adjust_timestamps(
                oldest_starttime, meas.metadata.starttime
            )
            meas.metadata.endtime = meas.metadata.starttime + durration

    return meas_list

In [92]:
# Sample data
import numpy as np
from datetime import datetime

# Mock Meas class (assuming it's already imported from src.classes)
# Sample data for testing with specific conditions
meas_list = [
    Meas(
        x_data=np.array([1, 2, 3]),  # Example x_data
        y_data=np.array([4, 5, 6]),  # Example y_data
        meas_number=1,
        condition="conditionA",
        gender="M",
        pair_number=1,
        shift=0.0,
        starttime=datetime.fromisoformat(
            "2024-12-15T12:00:30.000000+01:00"
        ),  # Oldest starttime
        endtime=datetime.fromisoformat("2024-12-15T12:01:00.000000+01:00"),
    ),
    Meas(
        x_data=np.array([1, 2, 3]),
        y_data=np.array([7, 8, 9]),
        meas_number=1,
        condition="conditionA",
        gender="M",
        pair_number=1,
        shift=0.0,
        starttime=datetime.fromisoformat(
            "2024-12-15T12:01:30.000000+01:00"
        ),  # After the endtime of the first
        endtime=datetime.fromisoformat("2024-12-15T12:02:00.000000+01:00"),
    ),
    Meas(
        x_data=np.array([1, 2, 3]),
        y_data=np.array([10, 11, 12]),
        meas_number=1,
        condition="conditionA",
        gender="M",
        pair_number=1,
        shift=0.0,
        starttime=datetime.fromisoformat(
            "2024-12-15T12:02:30.000000+01:00"
        ),  # After the endtime of the second
        endtime=datetime.fromisoformat("2024-12-15T12:03:00.000000+01:00"),
    ),
    Meas(
        x_data=np.array([1, 2, 3]),  # Example x_data
        y_data=np.array([4, 5, 6]),  # Example y_data
        meas_number=1,
        condition="conditionA",
        gender="F",
        pair_number=1,
        shift=0.0,
        starttime=datetime.fromisoformat(
            "2024-12-15T12:00:30.500000+01:00"
        ),  # Oldest starttime
        endtime=datetime.fromisoformat("2024-12-15T12:01:00.500000+01:00"),
    ),
    Meas(
        x_data=np.array([1, 2, 3]),
        y_data=np.array([7, 8, 9]),
        meas_number=1,
        condition="conditionA",
        gender="F",
        pair_number=1,
        shift=0.0,
        starttime=datetime.fromisoformat(
            "2024-12-15T12:01:30.500000+01:00"
        ),  # After the endtime of the first
        endtime=datetime.fromisoformat("2024-12-15T12:02:00.500000+01:00"),
    ),
    Meas(
        x_data=np.array([1, 2, 3]),
        y_data=np.array([10, 11, 12]),
        meas_number=1,
        condition="conditionA",
        gender="F",
        pair_number=1,
        shift=0.0,
        starttime=datetime.fromisoformat(
            "2024-12-15T12:02:30.500000+01:00"
        ),  # After the endtime of the second
        endtime=datetime.fromisoformat("2024-12-15T12:03:00.500000+01:00"),
    ),
]

In [93]:
# Now you can call the reset_meas_time function to test it
copy_meas_list = copy.deepcopy(meas_list)
adjusted_meas_list = normalize_meas_time(copy_meas_list)

In [94]:
# Print original start and end times for verification
print("Original Meas List:")
for meas in meas_list:
    print(
        f"Meas: {meas}, "
        f"Start Time: {meas.metadata.starttime}, "
        f"End Time: {meas.metadata.endtime}"
    )

# Print adjusted start and end times for verification
print("\nAdjusted Meas List:")
for meas in adjusted_meas_list:
    print(
        f"Meas: {meas}, "
        f"Start Time: {meas.metadata.starttime}, "
        f"End Time: {meas.metadata.endtime}"
    )

Original Meas List:
Meas: 1conditionAM1_0.0, Start Time: 2024-12-15 12:00:30+01:00, End Time: 2024-12-15 12:01:00+01:00
Meas: 1conditionAM1_0.0, Start Time: 2024-12-15 12:01:30+01:00, End Time: 2024-12-15 12:02:00+01:00
Meas: 1conditionAM1_0.0, Start Time: 2024-12-15 12:02:30+01:00, End Time: 2024-12-15 12:03:00+01:00
Meas: 1conditionAF1_0.0, Start Time: 2024-12-15 12:00:30.500000+01:00, End Time: 2024-12-15 12:01:00.500000+01:00
Meas: 1conditionAF1_0.0, Start Time: 2024-12-15 12:01:30.500000+01:00, End Time: 2024-12-15 12:02:00.500000+01:00
Meas: 1conditionAF1_0.0, Start Time: 2024-12-15 12:02:30.500000+01:00, End Time: 2024-12-15 12:03:00.500000+01:00

Adjusted Meas List:
Meas: 1conditionAM1_0.0, Start Time: 1970-01-01 00:00:00, End Time: 1970-01-01 00:00:30
Meas: 1conditionAM1_0.0, Start Time: 1970-01-01 00:01:00, End Time: 1970-01-01 00:01:30
Meas: 1conditionAM1_0.0, Start Time: 1970-01-01 00:02:00, End Time: 1970-01-01 00:02:30
Meas: 1conditionAF1_0.0, Start Time: 1970-01-01 00:00